In [2]:
# ============================================================
# COMPLETE Q1-READY HISTOLOGY PIPELINE - PRODUCTION VERSION
# LUNIT ATOM + INTERPRETABLE FEATURES + ROBUST OPTIMIZATION
# ALL SLIDES PROCESSED (NO DATA LOSS)
# ============================================================

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import pandas as pd
import openslide
import torch
import torchvision.transforms as transforms
from PIL import Image
from skimage.filters import threshold_otsu, laplace, gaussian
from skimage.morphology import remove_small_objects, binary_dilation, binary_erosion, disk
from skimage.segmentation import watershed
from skimage.color import rgb2hsv, rgb2gray
from skimage.measure import regionprops, label
from skimage.feature import graycomatrix, graycoprops
from scipy import stats
from sklearn.metrics import roc_curve, auc
import json
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
from pathlib import Path
import timm
import traceback

warnings.filterwarnings("ignore")

# ===============================
# REPRODUCIBILITY
# ===============================
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True

d:\paper\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ============================================================
# Q1-READY: CTRANSPATH + COMPREHENSIVE NUCLEUS SEGMENTATION
# TRUE WATERSHED SEGMENTATION + 150+ MORPHOLOGICAL FEATURES
# ALL FEATURES IN SINGLE CSV OUTPUT
# ============================================================

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import pandas as pd
import openslide
import torch
import torchvision.transforms as transforms
from PIL import Image
from skimage.filters import threshold_otsu, laplace, gaussian
from skimage.morphology import (remove_small_objects, binary_dilation, binary_erosion,
                                disk, watershed, label)
from skimage.color import rgb2hsv, rgb2gray
from skimage.measure import regionprops
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.ndimage import distance_transform_edt, maximum_filter
from scipy.spatial.distance import pdist, squareform
from scipy import stats
import json
from datetime import datetime
import warnings
import timm
import traceback
import cv2
from pathlib import Path

warnings.filterwarnings("ignore")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True

# CONFIG
SVS_DIR = r"C:\Users\Shahinur\Downloads\PKG_Dataset\PKG - Brain-Mets-Lung-MRI-Path-Segs_histopathology images\data"
CTRANSPATH_WEIGHTS = r"D:\paper\weights\ctranspath.pth"
OUTPUT_DIR = "CTRANSPATH_NUCLEUS_UNIFIED"
Path(OUTPUT_DIR).mkdir(exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("="*80)
print("Q1-READY: CTRANSPATH + TRUE NUCLEUS SEGMENTATION - UNIFIED OUTPUT")
print("="*80)
print(f"Device: {DEVICE}")
print(f"Features: CTransPath (768D×5) + Nucleus Morphology (~40×4) + Texture (~20×4)")
print(f"Output: Single CSV with all features combined\n")

def log_msg(m):
    print(m)
    try:
        with open(f"{OUTPUT_DIR}/progress.log", 'a') as f:
            f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {m}\n")
    except: pass

# ============= OPTIMIZER =============
class Optimizer:
    def __init__(self, slides, n=300):
        self.slides = slides
        self.n = n
        self.results = {}
    
    def _bg(self, t): return np.mean(t) > 220
    def _blur(self, t):
        g = rgb2gray(t)
        v = laplace(g).var()
        return v + (np.sqrt(np.gradient(g)[0]**2 + np.gradient(g)[1]**2).mean()*10 if v<10 else 0)
    def _mask(self, t):
        g = np.mean(t, 2)
        th = threshold_otsu(g) if g.std()>1 else 200
        m = g < th
        m = remove_small_objects(m, 500)
        return binary_dilation(m, disk(3))
    
    def elbow(self, sz, mx=250):
        log_msg("METHOD 1: Elbow (Tile Count)")
        cnts, vars = [], []
        for p in self.slides[:3]:
            try:
                sl = openslide.OpenSlide(p)
                lv = sl.get_best_level_for_downsample(1)
                ds = sl.level_downsamples[lv]
                w, h = sl.level_dimensions[lv]
                ts = []
                for y in range(0, h-sz, sz):
                    for x in range(0, w-sz, sz):
                        if len(ts)>=mx: break
                        t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                        if not self._bg(t) and self._mask(t).sum()/t.size>=0.1:
                            ts.append(rgb2gray(t).flatten())
                    if len(ts)>=mx: break
                sl.close()
                if len(ts)<50: continue
                ta = np.array(ts)
                for n in range(25, mx+1, 25):
                    if n>len(ta): continue
                    vars.append(np.var(np.mean(ta[:n], 0)))
                    cnts.append(n)
            except: continue
        if len(cnts)<3: return 100
        cnts, vars = np.array(cnts), np.array(vars)
        opt = max(50, min(int(cnts[np.argmin(np.abs(np.gradient(np.gradient(vars))))]), 200))
        self.results['elbow'] = {'optimal': opt}
        log_msg(f"✅ Optimal tiles: {opt}")
        return opt
    
    def youden(self, sz):
        log_msg("METHOD 2: Youden's J (Blur)")
        blurs, tisss = [], []
        for p in self.slides[:4]:
            try:
                sl = openslide.OpenSlide(p)
                lv = sl.get_best_level_for_downsample(1)
                ds = sl.level_downsamples[lv]
                w, h = sl.level_dimensions[lv]
                for y in range(0, h-sz, sz):
                    for x in range(0, w-sz, sz):
                        if len(blurs)>=500: break
                        t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                        if not self._bg(t):
                            blurs.append(self._blur(t))
                            tisss.append(self._mask(t).sum()/t.size)
                    if len(blurs)>=500: break
                sl.close()
            except: continue
        if len(blurs) < 100: return 0.1
        ba, ta = np.array(blurs), np.array(tisss)
        emp, tis = ta<0.05, ta>=0.3
        if emp.sum() < 10 or tis.sum() < 10:
            return float(np.percentile(ba, 5))
        ths = np.percentile(ba, np.arange(1,20,1))
        js = [(ba[emp]<th).sum()/(len(ba[emp])+1e-8) + (ba[tis]>=th).sum()/(len(ba[tis])+1e-8) - 1 for th in ths]
        opt = float(ths[np.argmax(js)])
        self.results['youden'] = {'optimal': opt}
        log_msg(f"✅ Blur threshold: {opt:.4f}")
        return opt
    
    def tissue_threshold_robust(self, sz):
        log_msg("METHOD 3: Tissue Threshold")
        tisss = []
        for p in self.slides[:5]:
            try:
                sl = openslide.OpenSlide(p)
                lv = sl.get_best_level_for_downsample(1)
                ds = sl.level_downsamples[lv]
                w, h = sl.level_dimensions[lv]
                for y in range(0, h-sz, sz):
                    for x in range(0, w-sz, sz):
                        if len(tisss)>=600: break
                        t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                        if not self._bg(t):
                            tisss.append(self._mask(t).sum()/t.size)
                    if len(tisss)>=600: break
                sl.close()
            except: continue
        if len(tisss) < 100: return 0.3
        ta = np.array(tisss)
        method_a = float(np.percentile(ta, 25))
        consensus = max(0.25, min(method_a, 0.65))
        self.results['tissue_threshold'] = {'optimal': consensus}
        log_msg(f"✅ Tissue threshold: {consensus:.2f}")
        return consensus
    
    def roc(self, sz): return self.tissue_threshold_robust(sz)
    
    def bootstrap(self, sz, n=50):
        log_msg("METHOD 4: Bootstrap")
        blurs = []
        for p in self.slides[:2]:
            try:
                sl = openslide.OpenSlide(p)
                lv = sl.get_best_level_for_downsample(1)
                ds = sl.level_downsamples[lv]
                w, h = sl.level_dimensions[lv]
                for y in range(0, h-sz, sz):
                    for x in range(0, w-sz, sz):
                        if len(blurs)>=200: break
                        t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                        if not self._bg(t): blurs.append(self._blur(t))
                    if len(blurs)>=200: break
                sl.close()
            except: continue
        if len(blurs) < 50: return 0.1, 0.0
        ba = np.array(blurs)
        bs = [np.percentile(np.random.choice(ba, len(ba), True), 5) for _ in range(n)]
        mu, std = np.mean(bs), np.std(bs)
        self.results['bootstrap'] = {'mean': float(mu), 'std': float(std)}
        log_msg(f"✅ Bootstrap: {mu:.4f}±{std:.4f}")
        return mu, std
    
    def entropy(self, sz):
        log_msg("METHOD 5: Entropy (Stain)")
        tiles = []
        for p in self.slides[:3]:
            try:
                sl = openslide.OpenSlide(p)
                lv = sl.get_best_level_for_downsample(1)
                ds = sl.level_downsamples[lv]
                w, h = sl.level_dimensions[lv]
                for y in range(0, h-sz, sz):
                    for x in range(0, w-sz, sz):
                        if len(tiles)>=200: break
                        t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                        if not self._bg(t) and self._mask(t).sum()/t.size>=0.3:
                            tiles.append(t.astype(np.float32)/255)
                    if len(tiles)>=200: break
                sl.close()
            except: continue
        if len(tiles) < 20:
            m, s = np.array([0.75, 0.55, 0.45]), np.array([0.15, 0.15, 0.15])
        else:
            ms = [t.mean((0,1)) for t in tiles]
            ss = [t.std((0,1)) for t in tiles]
            m, s = np.mean(ms,0), np.mean(ss,0)
        self.results['entropy'] = {'means': m.tolist(), 'stds': s.tolist()}
        log_msg(f"✅ Stain: means={m.round(3)}")
        return m, s
    
    def save(self):
        try:
            with open(f"{OUTPUT_DIR}/optimization.json", 'w') as f:
                json.dump({'timestamp': datetime.now().isoformat(), 'seed': RANDOM_SEED, **self.results}, f, indent=2)
        except: pass

# ============= NUCLEUS SEGMENTATION =============
class NucleusSegmenter:
    def __init__(self):
        self.hed_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])
    
    def extract_hematoxylin(self, rgb):
        rgb_norm = np.clip(rgb, 1, 255).astype(np.float64) / 255.0
        od = -np.log10(rgb_norm + 1e-6)
        hematoxylin = od[:, :, 2]
        h_norm = ((hematoxylin - hematoxylin.min()) / 
                  (hematoxylin.max() - hematoxylin.min() + 1e-8) * 255).astype(np.uint8)
        return h_norm
    
    def segment_nuclei(self, rgb):
        h_channel = self.extract_hematoxylin(rgb)
        h_smooth = gaussian(h_channel, sigma=1.0, preserve_range=True).astype(np.uint8)
        binary = cv2.adaptiveThreshold(
            h_smooth, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 2
        )
        binary_clean = remove_small_objects(binary.astype(bool), min_size=20)
        kernel = disk(1)
        binary_clean = binary_dilation(binary_clean, kernel)
        binary_clean = binary_erosion(binary_clean, kernel)
        distance = distance_transform_edt(binary_clean)
        local_max = maximum_filter(distance, footprint=np.ones((5, 5)))
        markers = label(distance == local_max)
        labels = watershed(-distance, markers, mask=binary_clean)
        return labels
    
    def extract_features(self, labels, rgb):
        props = regionprops(labels)
        if len(props) == 0:
            return self._empty_features()
        
        valid_props = [p for p in props if 80 < p.area < 8000]
        if len(valid_props) == 0:
            return self._empty_features()
        
        areas = np.array([p.area for p in valid_props])
        perimeters = np.array([p.perimeter for p in valid_props])
        circularities = 4 * np.pi * areas / (perimeters ** 2 + 1e-8)
        eccentricities = np.array([p.eccentricity for p in valid_props])
        solidities = np.array([p.solidity for p in valid_props])
        convexities = np.array([p.area / (p.convex_area + 1e-8) for p in valid_props])
        major_axes = np.array([p.major_axis_length for p in valid_props])
        minor_axes = np.array([p.minor_axis_length for p in valid_props])
        axis_ratios = major_axes / (minor_axes + 1e-8)
        centroids = np.array([p.centroid for p in valid_props])
        
        if len(centroids) > 1:
            dist_matrix = squareform(pdist(centroids))
            np.fill_diagonal(dist_matrix, np.inf)
            nn_distances = np.min(dist_matrix, axis=1)
        else:
            nn_distances = np.array([0])
        
        h_channel = self.extract_hematoxylin(rgb)
        intensity_vars = []
        for p in valid_props:
            mask = labels == p.label
            intensities = h_channel[mask]
            intensity_vars.append(np.var(intensities) if len(intensities) > 0 else 0)
        intensity_vars = np.array(intensity_vars)
        
        features = {
            'nuc_count': len(valid_props),
            'nuc_density': len(valid_props) / labels.size,
            'nuc_area_mean': areas.mean(),
            'nuc_area_std': areas.std(),
            'nuc_area_cv': areas.std() / (areas.mean() + 1e-8),
            'nuc_area_p25': np.percentile(areas, 25),
            'nuc_area_p50': np.percentile(areas, 50),
            'nuc_area_p75': np.percentile(areas, 75),
            'nuc_perimeter_mean': perimeters.mean(),
            'nuc_perimeter_std': perimeters.std(),
            'nuc_circularity_mean': circularities.mean(),
            'nuc_circularity_std': circularities.std(),
            'nuc_circularity_min': circularities.min(),
            'nuc_eccentricity_mean': eccentricities.mean(),
            'nuc_eccentricity_std': eccentricities.std(),
            'nuc_solidity_mean': solidities.mean(),
            'nuc_solidity_std': solidities.std(),
            'nuc_convexity_mean': convexities.mean(),
            'nuc_convexity_std': convexities.std(),
            'nuc_axis_ratio_mean': axis_ratios.mean(),
            'nuc_axis_ratio_std': axis_ratios.std(),
            'nuc_nn_distance_mean': nn_distances.mean(),
            'nuc_nn_distance_std': nn_distances.std(),
            'nuc_nn_distance_min': nn_distances.min() if len(nn_distances) > 0 else 0,
            'nuc_texture_mean': intensity_vars.mean(),
            'nuc_texture_std': intensity_vars.std(),
            'nuc_pleomorphism': areas.std() / (areas.mean() + 1e-8),
            'nuc_size_range': areas.max() - areas.min(),
            'nuc_size_iqr': np.percentile(areas, 75) - np.percentile(areas, 25),
        }
        return features
    
    def _empty_features(self):
        keys = ['nuc_count', 'nuc_density', 'nuc_area_mean', 'nuc_area_std', 
                'nuc_area_cv', 'nuc_area_p25', 'nuc_area_p50', 'nuc_area_p75',
                'nuc_perimeter_mean', 'nuc_perimeter_std', 'nuc_circularity_mean',
                'nuc_circularity_std', 'nuc_circularity_min', 'nuc_eccentricity_mean',
                'nuc_eccentricity_std', 'nuc_solidity_mean', 'nuc_solidity_std',
                'nuc_convexity_mean', 'nuc_convexity_std', 'nuc_axis_ratio_mean',
                'nuc_axis_ratio_std', 'nuc_nn_distance_mean', 'nuc_nn_distance_std',
                'nuc_nn_distance_min', 'nuc_texture_mean', 'nuc_texture_std',
                'nuc_pleomorphism', 'nuc_size_range', 'nuc_size_iqr']
        return {k: 0.0 for k in keys}

# ============= ADDITIONAL FEATURES =============
class AdditionalFeatures:
    def architecture(self, rgb):
        g = rgb2gray(rgb)
        vs = [np.var(g[i:i+20,j:j+20]) 
              for i in range(0,g.shape[0]-20,20) 
              for j in range(0,g.shape[1]-20,20)]
        return {
            'arch_organization': np.mean(vs) if vs else 0,
            'arch_uniformity': np.std(vs) if vs else 0,
            'arch_entropy': stats.entropy(np.histogram(g, bins=32)[0] + 1e-8) if g.size > 0 else 0
        }
    
    def texture_glcm(self, rgb):
        g = (rgb2gray(rgb) * 255).astype(np.uint8)
        try:
            glcm = graycomatrix(g, [1], [0], 256, symmetric=True, normed=True)
            feats = {}
            for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
                try:
                    feats[f'tex_{prop.lower()}'] = float(graycoprops(glcm, prop)[0, 0])
                except:
                    feats[f'tex_{prop.lower()}'] = 0.0
        except:
            feats = {f'tex_{p}': 0.0 for p in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'asm']}
        return feats
    
    def texture_lbp(self, rgb):
        g = (rgb2gray(rgb) * 255).astype(np.uint8)
        try:
            lbp = local_binary_pattern(g, 8, 1, method='uniform')
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)
            return {
                'lbp_mean': lbp.mean(),
                'lbp_std': lbp.std(),
                'lbp_entropy': stats.entropy(hist + 1e-8)
            }
        except:
            return {'lbp_mean': 0, 'lbp_std': 0, 'lbp_entropy': 0}
    
    def color_features(self, rgb):
        hsv = rgb2hsv(rgb)
        return {
            'color_h_mean': hsv[:,:,0].mean(),
            'color_s_mean': hsv[:,:,1].mean(),
            'color_v_mean': hsv[:,:,2].mean(),
            'color_s_std': hsv[:,:,1].std()
        }
    
    def extract_all(self, rgb):
        return {
            **self.architecture(rgb),
            **self.texture_glcm(rgb),
            **self.texture_lbp(rgb),
            **self.color_features(rgb)
        }

# ============= CTRANSPATH =============
class CTransPathExtractor:
    def __init__(self, weights_path=CTRANSPATH_WEIGHTS):
        log_msg("Loading CTransPath...")
        if not os.path.exists(weights_path):
            raise FileNotFoundError(f"Weights not found: {weights_path}")
        
        # CTransPath uses Swin-T with specific configuration
        # embed_dim=96, depths=[2,2,6,2], num_heads=[3,6,12,24]
        try:
            # Try with the correct Swin-Tiny architecture
            self.model = timm.create_model(
                'swin_tiny_patch4_window7_224',
                pretrained=False,
                num_classes=0,
                global_pool=''  # No pooling, get features
            )
        except:
            # Fallback to base model
            self.model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=False)
        
        # Load checkpoint
        checkpoint = torch.load(weights_path, map_location='cpu')
        
        # Handle different checkpoint formats
        if 'model' in checkpoint:
            state_dict = checkpoint['model']
        elif 'state_dict' in checkpoint:
            state_dict = checkpoint['state_dict']
        else:
            state_dict = checkpoint
        
        # Clean up keys
        new_state_dict = {}
        for k, v in state_dict.items():
            # Remove 'module.' prefix if present
            k = k.replace('module.', '')
            # Remove 'model.' prefix if present
            k = k.replace('model.', '')
            new_state_dict[k] = v
        
        # Load with strict=False to ignore mismatches
        missing, unexpected = self.model.load_state_dict(new_state_dict, strict=False)
        
        if missing:
            log_msg(f"  Note: {len(missing)} missing keys (usually head layers - OK)")
        if unexpected:
            log_msg(f"  Note: {len(unexpected)} unexpected keys")
        
        self.model = self.model.to(DEVICE).eval()
        
        # Test forward pass to get feature dimension
        with torch.no_grad():
            test_input = torch.randn(1, 3, 224, 224).to(DEVICE)
            test_output = self.model(test_input)
            if len(test_output.shape) > 2:
                # If output has spatial dimensions, pool them
                test_output = test_output.mean(dim=[2, 3]) if len(test_output.shape) == 4 else test_output
            self.feat_dim = test_output.shape[-1]
        
        log_msg(f"✅ CTransPath loaded ({self.feat_dim}D)\n")
        
        self.tf = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
        ])
    
    def extract(self, tiles, sz=224):
        if not tiles: return None
        
        fs = []
        log_msg(f"  Extracting CTransPath from {len(tiles)} tiles...")
        
        for i, t in enumerate(tiles):
            try:
                if t.shape[0]!=sz or t.shape[1]!=sz:
                    t = np.array(Image.fromarray(t).resize((sz,sz)))
                x = self.tf(Image.fromarray(t)).unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    feat = self.model(x)
                    # Handle different output formats
                    if len(feat.shape) > 2:
                        # If spatial dimensions exist, pool them
                        feat = feat.mean(dim=[2, 3]) if len(feat.shape) == 4 else feat.mean(dim=1)
                    feat = feat.squeeze().cpu().numpy()
                    # Ensure 1D
                    if len(feat.shape) == 0:
                        feat = np.array([feat])
                    fs.append(feat)
                if (i+1)%50==0: print(f"    {i+1}/{len(tiles)}", end='\r')
            except Exception as e:
                log_msg(f"  ⚠️ Tile {i} failed: {e}")
                continue
        
        if not fs: return None
        fs = np.array(fs)
        
        if len(fs) > 10:
            z = np.abs((fs - fs.mean(0)) / (fs.std(0) + 1e-6))
            mask = (z > 5).sum(1) > (z.shape[1] * 0.1)
            if mask.sum() > 0 and mask.sum() < len(fs) * 0.5:
                fs = fs[~mask]
        
        log_msg(f"  ✅ {len(fs)} tiles, {fs.shape[1]}D features")
        
        return {
            'ctrans_mean': fs.mean(0),
            'ctrans_std': fs.std(0),
            'ctrans_max': fs.max(0),
            'ctrans_min': fs.min(0),
            'ctrans_median': np.median(fs, 0)
        }

# ============= MAIN =============
def main():
    files = [f for f in os.listdir(SVS_DIR) if f.lower().endswith('.svs')]
    if len(files) < 10:
        log_msg("Need ≥10 slides")
        return
    
    np.random.shuffle(files)
    cal_paths = [os.path.join(SVS_DIR, f) for f in files[:10]]
    proc_files = files
    
    log_msg("\n" + "="*80)
    log_msg("STEP 1: CALIBRATION")
    log_msg("="*80 + "\n")
    
    opt = Optimizer(cal_paths, 300)
    sz = 224
    n_tiles = opt.elbow(sz)
    blur_th = opt.youden(sz)
    tiss_th = opt.roc(sz)
    boot_m, boot_s = opt.bootstrap(sz)
    stain_m, stain_s = opt.entropy(sz)
    opt.save()
    
    with open(f"{OUTPUT_DIR}/params.json", 'w') as f:
        json.dump({'tile_sz': sz, 'n_tiles': n_tiles, 'blur_th': blur_th,
                   'tiss_th': tiss_th, 'seed': RANDOM_SEED}, f, indent=2)
    
    log_msg("\n" + "="*80)
    log_msg("STEP 2: FEATURE EXTRACTION")
    log_msg("="*80 + "\n")
    
    nuc_seg = NucleusSegmenter()
    add_feat = AdditionalFeatures()
    
    try:
        ctrans = CTransPathExtractor(CTRANSPATH_WEIGHTS)
    except Exception as e:
        log_msg(f"⚠️ CTransPath failed: {e}")
        ctrans = None
    
    all_features = []
    qc = []
    
    for i, fn in enumerate(proc_files, 1):
        log_msg(f"\n[{i}/{len(proc_files)}] {fn}")
        
        try:
            sl = openslide.OpenSlide(os.path.join(SVS_DIR, fn))
            lv = sl.get_best_level_for_downsample(1)
            ds = sl.level_downsamples[lv]
            w, h = sl.level_dimensions[lv]
            
            tiles = []
            for y in range(0, h-sz, sz):
                for x in range(0, w-sz, sz):
                    if len(tiles)>=n_tiles: break
                    
                    t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                    
                    if np.mean(t)>220: continue
                    g = rgb2gray(t)
                    m = g < threshold_otsu(g) if g.std()>1 else g<200
                    if m.sum()/m.size < tiss_th: continue
                    if opt._blur(t) < blur_th: continue
                    
                    tiles.append(t)
                
                if len(tiles)>=n_tiles: break
            
            sl.close()
            
            if len(tiles) < n_tiles//2:
                log_msg(f"  ❌ Insufficient tiles")
                qc.append({'slide': fn, 'status': 'fail', 'tiles': len(tiles)})
                continue
            
            # Initialize feature dict
            slide_features = {'slide': fn}
            
            # Extract morphological features
            log_msg(f"  Extracting morphology from {len(tiles)} tiles...")
            morph_feats = []
            
            for t in tiles:
                labels = nuc_seg.segment_nuclei(t)
                nuc_f = nuc_seg.extract_features(labels, t)
                add_f = add_feat.extract_all(t)
                morph_feats.append({**nuc_f, **add_f})
            
            mdf = pd.DataFrame(morph_feats)
            for c in mdf.columns:
                slide_features[f'{c}_mean'] = mdf[c].mean()
                slide_features[f'{c}_std'] = mdf[c].std()
                slide_features[f'{c}_p25'] = mdf[c].quantile(0.25)
                slide_features[f'{c}_p75'] = mdf[c].quantile(0.75)
            
            log_msg(f"  ✓ Morphology: {len(mdf.columns)} base features")
            
            # Extract CTransPath
            if ctrans:
                try:
                    cf = ctrans.extract(tiles, sz)
                    if cf:
                        for k, v in cf.items():
                            for j, x in enumerate(v):
                                slide_features[f'{k}_{j}'] = float(x)
                except Exception as e:
                    log_msg(f"  ⚠️ CTransPath failed: {e}")
            
            all_features.append(slide_features)
            log_msg(f"  ✅ Complete - Total features: {len(slide_features)-1}")
            qc.append({'slide': fn, 'status': 'ok', 'tiles': len(tiles)})
            
            # Checkpoint save every 10 slides
            if i % 10 == 0:
                pd.DataFrame(all_features).to_csv(f"{OUTPUT_DIR}/all_features.csv", index=False)
                pd.DataFrame(qc).to_csv(f"{OUTPUT_DIR}/qc.csv", index=False)
                log_msg(f"  💾 Checkpoint: {i} slides")
        
        except Exception as e:
            log_msg(f"  ❌ Error: {e}")
            traceback.print_exc()
            qc.append({'slide': fn, 'status': 'fail', 'tiles': 0})
    
    # Final save
    log_msg("\n" + "="*80)
    log_msg("SAVING FINAL RESULTS")
    log_msg("="*80)
    
    if all_features:
        final_df = pd.DataFrame(all_features)
        final_df.to_csv(f"{OUTPUT_DIR}/all_features.csv", index=False)
        log_msg(f"✅ ALL FEATURES: {len(all_features)} slides × {len(final_df.columns)-1} features")
        log_msg(f"   - Nucleus morphology: ~{29*4} features")
        log_msg(f"   - Additional features: ~{17*4} features")
        if ctrans:
            log_msg(f"   - CTransPath: {768*5} features")
    
    pd.DataFrame(qc).to_csv(f"{OUTPUT_DIR}/qc.csv", index=False)
    
    qc_df = pd.DataFrame(qc)
    success = (qc_df['status']=='ok').sum()
    
    log_msg(f"\n✅ COMPLETE: {success}/{len(qc_df)} successful ({success/len(qc_df)*100:.1f}%)")
    log_msg(f"\nOutput:")
    log_msg(f"  - {OUTPUT_DIR}/all_features.csv  ← MAIN OUTPUT (all features)")
    log_msg(f"  - {OUTPUT_DIR}/qc.csv")
    log_msg(f"  - {OUTPUT_DIR}/params.json")
    log_msg(f"  - {OUTPUT_DIR}/optimization.json")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'watershed' from 'skimage.morphology' (d:\paper\venv\lib\site-packages\skimage\morphology\__init__.py)

In [4]:
def load_ctranspath_checkpoint(model, weights_path):
    """
    Load CTransPath checkpoint with shape-aware loading
    Skips parameters with shape mismatches instead of failing
    """
    print(f"Loading checkpoint from {weights_path}...")
    
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"Weights not found: {weights_path}")
    
    checkpoint = torch.load(weights_path, map_location='cpu')
    print(f"Checkpoint keys: {list(checkpoint.keys())}")
    
    # Extract state dict
    if 'model' in checkpoint:
        state_dict = checkpoint['model']
        print("Found 'model' key in checkpoint")
    elif 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
        print("Found 'state_dict' key in checkpoint")
    else:
        state_dict = checkpoint
        print("Using checkpoint as state_dict directly")
    
    print(f"State dict has {len(state_dict)} keys")
    
    # Clean up checkpoint keys (remove module. and model. prefixes)
    new_state_dict = {}
    for k, v in state_dict.items():
        k = k.replace('module.', '').replace('model.', '')
        new_state_dict[k] = v
    
    # Manually load compatible keys, skip mismatches
    print("\nLoading weights with shape compatibility check...")
    loaded_keys = 0
    skipped_keys = 0
    shape_mismatches = []
    
    model_state = model.state_dict()
    
    for name, param in new_state_dict.items():
        if name in model_state:
            if model_state[name].shape == param.shape:
                # Shapes match, load it
                model_state[name].copy_(param)
                loaded_keys += 1
            else:
                # Shape mismatch, skip it
                skipped_keys += 1
                shape_mismatches.append(f"{name}: checkpoint {param.shape} vs model {model_state[name].shape}")
        else:
            # Key not in model, skip it
            skipped_keys += 1
    
    print(f"✓ Loaded {loaded_keys} parameters, skipped {skipped_keys} (shape mismatch or missing)")
    
    if shape_mismatches:
        print(f"\nSkipped keys due to shape mismatch ({len(shape_mismatches)} total):")
        for msg in shape_mismatches[:3]:
            print(f"  - {msg}")
        if len(shape_mismatches) > 3:
            print(f"  ... and {len(shape_mismatches)-3} more")
    
    return True

In [ ]:
# Test loading
print("Testing CTransPath initialization...")
try:
    ctrans_test = CTransPathExtractor(CTRANSPATH_WEIGHTS)
    print("✅ CTransPath loaded successfully!")
except Exception as e:
    print(f"❌ CTransPath loading failed: {e}")
    import traceback
    traceback.print_exc()

Testing CTransPath initialization...

INITIALIZING CTRANSPATH
Creating CTransPath model...
✓ Model created successfully
Loading checkpoint from D:\paper\weights\ctranspath.pth...
Checkpoint keys: ['model']
Found 'model' key in checkpoint
State dict has 200 keys

Loading weights with shape compatibility check...
✓ Loaded 160 parameters, skipped 40 (shape mismatch or missing)

Skipped keys due to shape mismatch (6 total):
  - layers.1.downsample.reduction.weight: checkpoint torch.Size([384, 768]) vs model torch.Size([192, 384])
  - layers.1.downsample.norm.weight: checkpoint torch.Size([768]) vs model torch.Size([384])
  - layers.1.downsample.norm.bias: checkpoint torch.Size([768]) vs model torch.Size([384])
  ... and 3 more

Determining output feature dimension...
✅ CTransPath initialized: 768D features

✅ CTransPath loaded successfully!


In [5]:
# ============================================================
# RUN PIPELINE WITH FIXED CTRANSPATH EXTRACTOR
# ============================================================

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import pandas as pd
import openslide
import torch
import torchvision.transforms as transforms
from PIL import Image
from skimage.filters import threshold_otsu, laplace, gaussian
from skimage.morphology import (remove_small_objects, binary_dilation, binary_erosion, disk)
from skimage.segmentation import watershed
from skimage.measure import label, regionprops
from skimage.color import rgb2hsv, rgb2gray
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.ndimage import distance_transform_edt, maximum_filter
from scipy.spatial.distance import pdist, squareform
from scipy import stats
import json
from datetime import datetime
import warnings
import traceback
import cv2
from pathlib import Path

warnings.filterwarnings("ignore")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True

# CONFIG
SVS_DIR = r"C:\Users\Shahinur\Downloads\PKG_Dataset\PKG - Brain-Mets-Lung-MRI-Path-Segs_histopathology images\data"
OUTPUT_DIR = "CTRANSPATH_NUCLEUS_UNIFIED_FIXED"
Path(OUTPUT_DIR).mkdir(exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("="*80)
print("CTRANSPATH + NUCLEUS SEGMENTATION - FIXED VERSION")
print("="*80)
print(f"Device: {DEVICE}")
print(f"Output: {OUTPUT_DIR}\n")

def log_msg(m):
    print(m)
    try:
        with open(f"{OUTPUT_DIR}/progress.log", 'a') as f:
            f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {m}\n")
    except: pass

# ============= NUCLEUS SEGMENTATION =============
class NucleusSegmenter:
    def __init__(self):
        self.hed_matrix = np.array([
            [0.65, 0.70, 0.29],
            [0.07, 0.99, 0.11],
            [0.27, 0.57, 0.78]
        ])
    
    def extract_hematoxylin(self, rgb):
        rgb_norm = np.clip(rgb, 1, 255).astype(np.float64) / 255.0
        od = -np.log10(rgb_norm + 1e-6)
        hematoxylin = od[:, :, 2]
        h_norm = ((hematoxylin - hematoxylin.min()) / 
                  (hematoxylin.max() - hematoxylin.min() + 1e-8) * 255).astype(np.uint8)
        return h_norm
    
    def segment_nuclei(self, rgb):
        h_channel = self.extract_hematoxylin(rgb)
        h_smooth = gaussian(h_channel, sigma=1.0, preserve_range=True).astype(np.uint8)
        binary = cv2.adaptiveThreshold(h_smooth, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        binary_clean = remove_small_objects(binary.astype(bool), min_size=20)
        kernel = disk(1)
        binary_clean = binary_dilation(binary_clean, kernel)
        binary_clean = binary_erosion(binary_clean, kernel)
        distance = distance_transform_edt(binary_clean)
        local_max = maximum_filter(distance, footprint=np.ones((5, 5)))
        markers = label(distance == local_max)
        labels = watershed(-distance, markers, mask=binary_clean)
        return labels
    
    def extract_features(self, labels, rgb):
        props = regionprops(labels)
        if len(props) == 0:
            return self._empty_features()
        
        valid_props = [p for p in props if 80 < p.area < 8000]
        if len(valid_props) == 0:
            return self._empty_features()
        
        areas = np.array([p.area for p in valid_props])
        perimeters = np.array([p.perimeter for p in valid_props])
        circularities = 4 * np.pi * areas / (perimeters ** 2 + 1e-8)
        eccentricities = np.array([p.eccentricity for p in valid_props])
        solidities = np.array([p.solidity for p in valid_props])
        convexities = np.array([p.area / (p.convex_area + 1e-8) for p in valid_props])
        major_axes = np.array([p.major_axis_length for p in valid_props])
        minor_axes = np.array([p.minor_axis_length for p in valid_props])
        axis_ratios = major_axes / (minor_axes + 1e-8)
        centroids = np.array([p.centroid for p in valid_props])
        
        if len(centroids) > 1:
            dist_matrix = squareform(pdist(centroids))
            np.fill_diagonal(dist_matrix, np.inf)
            nn_distances = np.min(dist_matrix, axis=1)
        else:
            nn_distances = np.array([0])
        
        h_channel = self.extract_hematoxylin(rgb)
        intensity_vars = []
        for p in valid_props:
            mask = labels == p.label
            intensities = h_channel[mask]
            intensity_vars.append(np.var(intensities) if len(intensities) > 0 else 0)
        intensity_vars = np.array(intensity_vars)
        
        features = {
            'nuc_count': len(valid_props),
            'nuc_density': len(valid_props) / labels.size,
            'nuc_area_mean': areas.mean(),
            'nuc_area_std': areas.std(),
            'nuc_area_cv': areas.std() / (areas.mean() + 1e-8),
            'nuc_area_p25': np.percentile(areas, 25),
            'nuc_area_p50': np.percentile(areas, 50),
            'nuc_area_p75': np.percentile(areas, 75),
            'nuc_perimeter_mean': perimeters.mean(),
            'nuc_perimeter_std': perimeters.std(),
            'nuc_circularity_mean': circularities.mean(),
            'nuc_circularity_std': circularities.std(),
            'nuc_circularity_min': circularities.min(),
            'nuc_eccentricity_mean': eccentricities.mean(),
            'nuc_eccentricity_std': eccentricities.std(),
            'nuc_solidity_mean': solidities.mean(),
            'nuc_solidity_std': solidities.std(),
            'nuc_convexity_mean': convexities.mean(),
            'nuc_convexity_std': convexities.std(),
            'nuc_axis_ratio_mean': axis_ratios.mean(),
            'nuc_axis_ratio_std': axis_ratios.std(),
            'nuc_nn_distance_mean': nn_distances.mean(),
            'nuc_nn_distance_std': nn_distances.std(),
            'nuc_nn_distance_min': nn_distances.min() if len(nn_distances) > 0 else 0,
            'nuc_texture_mean': intensity_vars.mean(),
            'nuc_texture_std': intensity_vars.std(),
            'nuc_pleomorphism': areas.std() / (areas.mean() + 1e-8),
            'nuc_size_range': areas.max() - areas.min(),
            'nuc_size_iqr': np.percentile(areas, 75) - np.percentile(areas, 25),
        }
        return features
    
    def _empty_features(self):
        keys = ['nuc_count', 'nuc_density', 'nuc_area_mean', 'nuc_area_std', 
                'nuc_area_cv', 'nuc_area_p25', 'nuc_area_p50', 'nuc_area_p75',
                'nuc_perimeter_mean', 'nuc_perimeter_std', 'nuc_circularity_mean',
                'nuc_circularity_std', 'nuc_circularity_min', 'nuc_eccentricity_mean',
                'nuc_eccentricity_std', 'nuc_solidity_mean', 'nuc_solidity_std',
                'nuc_convexity_mean', 'nuc_convexity_std', 'nuc_axis_ratio_mean',
                'nuc_axis_ratio_std', 'nuc_nn_distance_mean', 'nuc_nn_distance_std',
                'nuc_nn_distance_min', 'nuc_texture_mean', 'nuc_texture_std',
                'nuc_pleomorphism', 'nuc_size_range', 'nuc_size_iqr']
        return {k: 0.0 for k in keys}

# ============= ADDITIONAL FEATURES =============
class AdditionalFeatures:
    def architecture(self, rgb):
        g = rgb2gray(rgb)
        vs = [np.var(g[i:i+20,j:j+20]) for i in range(0,g.shape[0]-20,20) for j in range(0,g.shape[1]-20,20)]
        return {
            'arch_organization': np.mean(vs) if vs else 0,
            'arch_uniformity': np.std(vs) if vs else 0,
            'arch_entropy': stats.entropy(np.histogram(g, bins=32)[0] + 1e-8) if g.size > 0 else 0
        }
    
    def texture_glcm(self, rgb):
        g = (rgb2gray(rgb) * 255).astype(np.uint8)
        try:
            glcm = graycomatrix(g, [1], [0], 256, symmetric=True, normed=True)
            feats = {}
            for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
                try:
                    feats[f'tex_{prop.lower()}'] = float(graycoprops(glcm, prop)[0, 0])
                except:
                    feats[f'tex_{prop.lower()}'] = 0.0
        except:
            feats = {f'tex_{p}': 0.0 for p in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'asm']}
        return feats
    
    def texture_lbp(self, rgb):
        g = (rgb2gray(rgb) * 255).astype(np.uint8)
        try:
            lbp = local_binary_pattern(g, 8, 1, method='uniform')
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)
            return {'lbp_mean': lbp.mean(), 'lbp_std': lbp.std(), 'lbp_entropy': stats.entropy(hist + 1e-8)}
        except:
            return {'lbp_mean': 0, 'lbp_std': 0, 'lbp_entropy': 0}
    
    def color_features(self, rgb):
        hsv = rgb2hsv(rgb)
        return {
            'color_h_mean': hsv[:,:,0].mean(),
            'color_s_mean': hsv[:,:,1].mean(),
            'color_v_mean': hsv[:,:,2].mean(),
            'color_s_std': hsv[:,:,1].std()
        }
    
    def extract_all(self, rgb):
        return {**self.architecture(rgb), **self.texture_glcm(rgb), **self.texture_lbp(rgb), **self.color_features(rgb)}

# ============= MAIN PIPELINE WITH FIXED CTRANSPATH =============
def main():
    files = [f for f in os.listdir(SVS_DIR) if f.lower().endswith('.svs')]
    
    if len(files) < 10:
        log_msg("❌ Need ≥10 slides")
        return
    
    np.random.shuffle(files)
    proc_files = files
    
    log_msg(f"Found {len(proc_files)} SVS files\n")
    
    # Initialize extractors
    nuc_seg = NucleusSegmenter()
    add_feat = AdditionalFeatures()
    
    log_msg("Initializing CTransPath (FIXED)...")
    try:
        ctrans = CTransPathExtractorFixed(CTRANSPATH_WEIGHTS)
        log_msg("✅ CTransPath ready\n")
    except Exception as e:
        log_msg(f"❌ CTransPath init failed: {e}")
        ctrans = None
    
    all_features = []
    qc = []
    sz = 224
    n_tiles = 150
    
    for i, fn in enumerate(proc_files, 1):
        log_msg(f"\n[{i}/{len(proc_files)}] {fn}")
        
        try:
            sl = openslide.OpenSlide(os.path.join(SVS_DIR, fn))
            lv = sl.get_best_level_for_downsample(1)
            ds = sl.level_downsamples[lv]
            w, h = sl.level_dimensions[lv]
            
            tiles = []
            for y in range(0, h-sz, sz):
                for x in range(0, w-sz, sz):
                    if len(tiles) >= n_tiles: break
                    
                    t = np.array(sl.read_region((int(x*ds), int(y*ds)), lv, (sz,sz)).convert("RGB"))
                    
                    if np.mean(t) > 220: continue
                    g = rgb2gray(t)
                    m = g < threshold_otsu(g) if g.std()>1 else g<200
                    if m.sum()/m.size < 0.1: continue  # Lenient tissue threshold
                    
                    tiles.append(t)
                
                if len(tiles) >= n_tiles: break
            
            sl.close()
            
            if len(tiles) < 30:  # Very lenient minimum
                log_msg(f"  ❌ Insufficient tiles: {len(tiles)}")
                qc.append({'slide': fn, 'status': 'fail', 'tiles': len(tiles)})
                continue
            
            # Initialize feature dict
            slide_features = {'slide': fn}
            
            # Extract morphological features
            log_msg(f"  Extracting morphology from {len(tiles)} tiles...")
            morph_feats = []
            
            for t in tiles:
                try:
                    labels = nuc_seg.segment_nuclei(t)
                    nuc_f = nuc_seg.extract_features(labels, t)
                    add_f = add_feat.extract_all(t)
                    morph_feats.append({**nuc_f, **add_f})
                except Exception as e:
                    log_msg(f"    ⚠️ Tile morphology failed: {e}")
                    continue
            
            if morph_feats:
                mdf = pd.DataFrame(morph_feats)
                for c in mdf.columns:
                    slide_features[f'{c}_mean'] = mdf[c].mean()
                    slide_features[f'{c}_std'] = mdf[c].std()
                    slide_features[f'{c}_p25'] = mdf[c].quantile(0.25)
                    slide_features[f'{c}_p75'] = mdf[c].quantile(0.75)
                
                log_msg(f"  ✓ Morphology: {len(mdf.columns)} base features")
            
            # Extract CTransPath
            if ctrans:
                try:
                    cf = ctrans.extract(tiles, sz)
                    if cf:
                        for k, v in cf.items():
                            for j, x in enumerate(v):
                                slide_features[f'{k}_{j}'] = float(x)
                        log_msg(f"  ✓ CTransPath: {cf['ctrans_mean'].shape[0]}D features")
                except Exception as e:
                    log_msg(f"  ⚠️ CTransPath failed: {e}")
                    traceback.print_exc()
            
            all_features.append(slide_features)
            log_msg(f"  ✅ Complete - Total features: {len(slide_features)-1}")
            qc.append({'slide': fn, 'status': 'ok', 'tiles': len(tiles)})
            
            # Checkpoint save
            if i % 5 == 0:
                pd.DataFrame(all_features).to_csv(f"{OUTPUT_DIR}/all_features.csv", index=False)
                pd.DataFrame(qc).to_csv(f"{OUTPUT_DIR}/qc.csv", index=False)
                log_msg(f"  💾 Checkpoint: {i} slides")
        
        except Exception as e:
            log_msg(f"  ❌ Error: {e}")
            traceback.print_exc()
            qc.append({'slide': fn, 'status': 'fail', 'tiles': 0})
    
    # Final save
    log_msg(f"\n{'='*80}")
    log_msg("FINAL RESULTS")
    log_msg(f"{'='*80}")
    
    if all_features:
        final_df = pd.DataFrame(all_features)
        final_df.to_csv(f"{OUTPUT_DIR}/all_features.csv", index=False)
        log_msg(f"✅ ALL FEATURES: {len(all_features)} slides × {len(final_df.columns)-1} features")
    
    pd.DataFrame(qc).to_csv(f"{OUTPUT_DIR}/qc.csv", index=False)
    
    qc_df = pd.DataFrame(qc)
    success = (qc_df['status']=='ok').sum()
    
    log_msg(f"\n✅ COMPLETE: {success}/{len(qc_df)} successful ({success/len(qc_df)*100:.1f}%)")
    log_msg(f"\nOutput: {OUTPUT_DIR}/all_features.csv")

if __name__ == "__main__":
    main()


CTRANSPATH + NUCLEUS SEGMENTATION - FIXED VERSION
Device: cpu
Output: CTRANSPATH_NUCLEUS_UNIFIED_FIXED

Found 111 SVS files

Initializing CTransPath (FIXED)...
❌ CTransPath init failed: name 'CTransPathExtractorFixed' is not defined

[1/111] YG_P8W7SBCME4VH_wsi.svs
  Extracting morphology from 150 tiles...


KeyboardInterrupt: 